# Equal-Weight S&P 500 Index Fund



The goal of is to create a Python script that will accept the value of a portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.


In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [ ]:
stocks = pd.read_csv('../sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': ')L/ LRNECSSDEMQN TT (GLAABESOAAKG', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'fcloifia', 'close': None, 'closeTime': None, 'closeSource': 'ilofcaif', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 130.01, 'latestSource': 'IEX real time price', 'latestTime': '10:55:45 AM', 'latestUpdate': 1686273655760, 'latestVolume': None, 'iexRealtimePrice': 132.93, 'iexRealtimeSize': 47, 'iexLastUpdated': 1666997853301, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 130.86, 'previousVolume': 115360722, 'change': -0.03, 'changePercent': -0.00024, 'volume': None, 'iexMarketPercent': 0.01251982318569605, 'iexVolume': 456812, 'avgTotalVolume': 113001741, 'ie

In [ ]:
price = data['latestPrice']
market_cap = data['marketCap']

In [ ]:
#### Adding Our Stocks Data to a Pandas DataFrame
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [ ]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,130.01,2183518918199,N/A


In [ ]:
# Looping Through The Tickers in Our List of Stocks
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True
    )

In [ ]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,133.030,39366872493,N/A
1,AAL,16.210,9815553653,N/A
2,AAP,167.076,11288595782,N/A
3,AAPL,129.780,2216217542006,N/A
4,ABBV,115.050,203980777067,N/A


In [ ]:
def chunks(lst, n):
    "Yield successive n-sized chunks from list."
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index=True
        )

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,130.346,40527794594,N/A
1,AAL,16.770,10017922788,N/A
2,AAP,174.092,11746396257,N/A
3,AAPL,131.400,2203652069531,N/A
4,ABBV,114.507,198528364449,N/A
...,...,...,...,...
500,YUM,112.210,33075398208,N/A
501,ZBH,161.590,33476578508,N/A
502,ZBRA,424.880,22123181524,N/A
503,ZION,49.730,8210756172,N/A


In [ ]:
# Calculating the Number of Shares to Buy
portfolio_size = input('Enter the value of your portfolio:')

while True:
    try:
        val = float(portfolio_size)
        print(val)
        break
    except ValueError:
        print('Please enter a number.')
        portfolio_size = input('Enter the value of your portfolio: ')

10000000.0


In [ ]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,130.346,40527794594,151
1,AAL,16.770,10017922788,1180
2,AAP,174.092,11746396257,113
3,AAPL,131.400,2203652069531,150
4,ABBV,114.507,198528364449,172
...,...,...,...,...
500,YUM,112.210,33075398208,176
501,ZBH,161.590,33476578508,122
502,ZBRA,424.880,22123181524,46
503,ZION,49.730,8210756172,398


In [ ]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

integer_format = writer.book.add_format(
{
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

Applying the Formats to the Columns of Our excel File


```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format],
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()